# โมเสคฝนรายรายชั่วโมงและความถี่สถานีเรดาร์ CBB+pulse volume เลือกเวลาโมเสคได้ อ่านจาก zip ไฟล์
* หาจำนวนสถานีที่จะใช้ในการโมเสคแต่ละชั่วโมง ทำลิสต์แต่ละการสแกนในชั่วโมงนั้น ๆ ไว้ เพื่อเรียกใช้ในการโมเสค ในที่นี้ได้สร้างโปรแกรมเพื่อสร้างลิสต์ไฟล์ไว้ก่อนหน้าแล้ว
* อ่านข้อมูลเรดาร์ zip ใน Pyart ใช้มุมยกแรกอย่างเดียว เนื่องต้องการให้ได้ฝนใกล้ภาคพื้นดินมากที่สุด
* ปรับแก้ noise ใน Pyart ด้วยวิธีการ Signal to Noise Ratio (SNR) ส่งออกเป็นไฟล์นามสกุล NetCDF ไปยัง wradlib
* อ่านไฟล์ NetCDF เข้ามาใน wradlib 
* ปรับแก้ clutter / attenuation ในระบบพิกัดโพลาร์ spherical coordinates  ทำการคำนวณ rainfall depth ของแต่ละสถานี ด้วย ZR แบบ Tropical Rosenfeld Z = 250R^1.2
* สร้างขอบเขตการโมเสคเรดาร์ในพื้นที่ศึกษา โดยใช้พิกัดดกริด UTM ด้วยการแปลงค่าพิกัดเรดาร์โพลาร์เป็นกริด utm 47 n ทุกสถานี โดยในท้ายที่สุดจะแปลงค่าพิกัดเหล่านั้นให้กลายเป็น geographic coordinates
* โมเสครายชั่วโมง ด้วยการลูปเพื่อโมเสคราย 15 นาที ด้วยการถ่วงค่าน้ำหนักตามดัชนีคุณภาพ (Quality Index) จากสองข้อมูล ได้แก่ ขนาดปริมาตรลำบีม (pulse volume) กับ ค่าการบดบังลำบีมสะสมวิเคราะห์ได้จากภูมิประเทศ (cumulative beam blockage: CBB) โดยค่า CBB หาได้จากการคำนวณ Patial Beam Blockage (PBB) จากแบบจำลองภูมิประเทศเชิงเลข (Digital Elevation Model: DEM) ที่ได้เตรียมคำนวณไว้ในโค้ดก่อนหน้านั้นแล้ว
* เก็บข้อมูลฝนโมเสคราย 15 นาทีไว้ใน data_list รายชั่วโมงนั้น ๆ แล้วทำการหาค่าหาฝนสะสมรายชั่วโมง (rainfall depth)
* สร้างไฟล์รายงานของสถานีที่ใช้ในการโมเสคในแต่ละ15นาทีดังกล่าว เก็บเอาไว้สะสมเป็นรายชั่วโมงนั้น ๆ เพื่อแสดงเป็นหนึ่งในคุณภาพข้อมูลประกอบผลโมเสครายชั่วโมงนั้น ๆ  ส่งออกเป็น *txt
* คำนวณแผนที่ความถี่ของสถานีเรดาร์ที่ใช้ในการโมเสคในแต่ละชั่วโมง 
* แปลงเวลา UTC เป็น Local time แล้วใช้ตั้งชื่อไฟล์
* ส่งออกผลลัพธ์ฝนสะสมรายชั่วโมงและความถี่สถานีเรดาร์ไปใน GIS ชนิด GeoTIFF

In [1]:
import numpy as np
import numpy.ma as ma
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as pl
import wradlib as wrl
from wradlib.io import read_generic_netcdf
import pyart
import sys
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import re
import time
from datetime import datetime
import pytz
import gzip
import shutil
import logging

print(wrl.__version__)
print(sys.version)
print(pyart.__version__)

#จับเวลา
# get the start time
st = time.time()

def extract_timestamps(input_path):
    # ช่วยอ่านไฟล์ที่จะนำมาใช้โมเสค
    timestamp_pattern = re.compile(r'(\d{10})')  # Matches YYYYMMDDhh
    timestamp_set = set()
    
    print(f"Scanning directory: {input_path}")
    for station in os.listdir(input_path):
        station_path = os.path.join(input_path, station)
        if os.path.isdir(station_path):
            print(f"Scanning station directory: {station}")
            for filename in os.listdir(station_path):
                print(f"  Processing file: {filename}")
                match = timestamp_pattern.search(filename)
                if match:
                    timestamp = match.group(1)
                    print(f"    Found timestamp: {timestamp}")
                    timestamp_set.add(timestamp)
                else:
                    print(f"    No timestamp found in filename: {filename}")
    
    timestamp_list = sorted(list(timestamp_set))
    print(f"Total unique timestamps found: {len(timestamp_list)}")
    return timestamp_list

def save_timestamp_list(timestamp_list, output_file):
    # เซฟไฟล์ชื่อชั่วโมงที่จะนำมาใช้โมเสค
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w') as f:
        for timestamp in timestamp_list:
            f.write(f"{timestamp}\n")
    print(f"Timestamp list saved to {output_file}")

def read_timestamp_list(input_file):
    # อ่านฟล์ชื่อชั่วโมงที่จะนำมาใช้โมเสค
    with open(input_file, 'r') as f:
        return [line.strip() for line in f]
    
def convert_utc_to_thailand_for_saving(utc_timestamp):
    """
    แปลงชื่อไฟล์จาก UTC ไปเป็น local time เวลาในไทย แล้วนำไปใช้ save เป็นชื่อไฟล์ไปเลย
    Convert UTC timestamp to Thailand local time (UTC+07) for file saving purposes.
    
    :param utc_timestamp: UTC timestamp string in format 'YYYYMMDDhh'
    :return: Thailand local time timestamp string in format 'YYYYMMDDhh'
    """
    utc_time = datetime.strptime(utc_timestamp, '%Y%m%d%H')
    utc_time = pytz.utc.localize(utc_time)
    thailand_time = utc_time.astimezone(pytz.timezone('Asia/Bangkok'))
    return thailand_time.strftime('%Y%m%d%H')    

# เพื่อแตกไฟล์ gzip ให้เป็น uf 
def decompress_gzip(input_file, output_file):
    try:
        with gzip.open(input_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"File decompressed successfully: {output_file}")
        return True
    except Exception as e:
        print(f"An error occurred while decompressing {input_file}: {str(e)}")
        return False

def check_and_decompress(path, mos_time):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.bz2') and mos_time in file:
                bz2_file = os.path.join(root, file)
                uf_file = os.path.splitext(bz2_file)[0]
                if not os.path.exists(uf_file):
                    print(f"Decompressing {bz2_file}")
                    decompress_gzip(bz2_file, uf_file)     

                    
def cleanup_files(base_path, folders=['CHN', 'CRI', 'LMP', 'PHS'], extensions_to_delete=['.uf', '.nc']):
    logging.info(f"Starting cleanup in {base_path}")
    
    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            logging.warning(f"Folder not found: {folder_path}")
            continue
        
        logging.info(f"Cleaning up folder: {folder_path}")
        for ext in extensions_to_delete:
            files_to_delete = glob.glob(os.path.join(folder_path, f'*{ext}'))
            for file in files_to_delete:
                try:
                    os.remove(file)
                    logging.info(f"Deleted: {file}")
                except Exception as e:
                    logging.error(f"Error deleting {file}: {str(e)}")
    
    logging.info("Cleanup completed")

# Set up logging
logging.basicConfig(filename='radar_cleanup.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')


def remove_noise_flare_SNR(file_in, file_out):
    #radar = pyart.io.read_uf(file_in)
    radar =pyart.io.auto_read.read(file_in)
    
    snr = pyart.retrieve.calculate_snr_from_reflectivity(radar, refl_field='reflectivity',toa=2500.0) #ค่าตั้งต้นคือ 15000 ทดลองเอง 
#ลอง 12500 ไม่เวิร์ค ลอง 5000 ก็ไม่เวิร์คเรดาร์ชุมพรฝนหาย ลอง 1000 เวิร์ค ฝนมี แต่เหมือนบางลง ตกลงใช้ toa 2500 เพราะฝนหายน้อยกว่า ได้ฝนครบ

    radar.add_field('signal_to_noise_ratio', snr, replace_existing=True)

    # กรองสัญญาณรบกวน ด้วยการใช้ SNR ใช้ gatefilter
    # ตรงนี้ต้องใช้ if กรณีที่ไม่มีค่า dual-pol ฟิวด์ ให้กลับไปใช้ SNR ที่คำนวณได้ ในการทำ texture
    gtfilter = pyart.filters.moment_and_texture_based_gate_filter(radar, phi_field='differential_phase')
    gtfilter.exclude_below('signal_to_noise_ratio', 2) #ใช้ค่า snr  = 10 
    gtfilter.exclude_above('signal_to_noise_ratio', 70) #ใช้ค่า snr  = 60 

    # แอดฟิวด์ที่ได้กรองสัญญาณรบกวนออกไป เอาโค้ดมาจาก  https://github.com/ARM-DOE/pyart/issues/763
    nf = radar.fields['corrected_reflectivity']
    nf['data'] = np.ma.masked_where(gtfilter.gate_excluded , nf['data'])
    radar.add_field('filtered_refectivity', nf, replace_existing=True)

    # ส่งออก radar object เพื่อนำฟิวด์ที่กรองสัญญาณรบกวนออกไปnetcdf เพื่อใช้ใน wradlib
    #file = './0data/PHI201807171100SNR'
    pyart.io.write_cfradial(file_out, radar, format='NETCDF4')

# ดัดแปลงโค้ดจาก https://docs.wradlib.org/en/latest/notebooks/workflow/recipe1.html
# ปรับแก้ clutter/attenuation คำนวณ rainfall depth
def correct_clutter_attenuation(vol, i=0):
    
    #print('Clutter + Attenuation + Rain estimates...............')
    
    # เก็บตัวแปร variables
    v = vol['variables']

    # อินเด็กซ์ของการกวาดในแต่ละมุมยก swstart=ค่าอินเด็กซ์แรกของแต่ละมุมยก swend= ค่าอินเด็กซ์สุดท้ายของแต่ละมุมยก
    swstart = v['sweep_start_ray_index']['data']
    swend = v['sweep_end_ray_index']['data']

     # ค่าเริ่มและหยุดในการกวาดมุมยกแรก i=0
    i = 0
    i1 = swstart[i]
    i2 = swend[i]

    # เก็บค่า dbz, azi, r มุมยกแรก [i1:i2]
    # ค่าการสะท้อน
    dbzh = v['corrected_reflectivity']['data']
    dbz=dbzh[i1:i2]
            
    #!!!!!! 
    # ทดสอบตัด clutter จะช่วย cri เพราะข้อมูลตั้งต้นมีค่าสูงกว่า 55 dbz ซึ่งผิดปรกติ 
    #ถ้าไม่ตัดค่าสูงนี้ออก จะได้ผลเป็น flare ที่เยอะมากใน svp ไม่เชื่อก็ลองcommentดู
    #dbz[(dbz<10) & (dbz>55)] = np.NaN
    #dbz[dbz==-9999.0]  = np.NaN
    #dbz=ma.masked_invalid(dbz)

    # ปรับแก้ clutter
    #--1.กำหนดตัวแปรหา clutter
    clutter = wrl.clutter.filter_gabella(dbz,    
                                      #wsize=5,
                                     #thrsnorain=10.0,
                                         tr1=12,
                                         n_p=6,
                                         tr2=1.1)
                                         #rm_nans=False) #เปลี่ยนจาก True
                                         #radial=False,
                                         #cartesian=False)
      #--2.ประมาณค่าบริเวณที่มีการลบ clutter
    data_no_clutter = wrl.ipol.interpolate_polar(dbz, clutter) # mask หรือ ลบค่าที่เป็น cluter ออก แล้วประมาณค่าเชิงพื้นที่กลับคืนไป

    #3. ปรับแก้ค่าการอ่อนสัญญาณ attenuation
    pia = wrl.atten.correct_attenuation_constrained(
                data_no_clutter,
                a_max=1.67e-4,
                a_min=2.33e-5,
                n_a=100,
                b_max=0.7,
                b_min=0.65,
                n_b=6,
                gate_length=1.0,
                constraints=[wrl.atten.constraint_dbz, wrl.atten.constraint_pia],
                constraint_args=[[59.0], [10.0]],
    )
    data_attcorr = data_no_clutter + pia

    # 4. converting to precipitation depth
    R = wrl.zr.z_to_r(wrl.trafo.idecibel(data_attcorr), a=256, b=1.4)
    depth = wrl.trafo.r_to_depth(R, 900.0) #900 = 15 นาที * 60 วินาที
    
    # เพิ่มเข้ามา เพื่อทดสอบจัดทำ mask สำหรับค่าน้อย ๆ ให้กลายเป็น nodata  อันนี้ต้องระวัง ระหว่าง nodata กับ ฝน 0 mm ไม่เหมือนกัน
    #depth[depth<0.01] = -9999 
    
    # Add this at the end to QC:
    depth[depth < 0] = np.nan  # Remove negative values
    depth[depth > 100] = np.nan  # Remove unrealistically high values
            
    return depth

#ฟังค์ชั่นรับค่าพารามิเตอร์เพื่อนำไปคำนวณพิกัด utm
def get_geom(vol, i=0) :
    # เก็บตัวแปร variables
    v = vol['variables']

    # เก็บพิกัด
    lon = v['longitude']['data']
    lat = v['latitude']['data']
    alt = v['altitude']['data']
    radar_location = (lon, lat)    
    
    # อินเด็กซ์ของการกวาดในแต่ละมุมยก swstart=ค่าอินเด็กซ์แรกของแต่ละมุมยก swend= ค่าอินเด็กซ์สุดท้ายของแต่ละมุมยก
    swstart = v['sweep_start_ray_index']['data']
    swend = v['sweep_end_ray_index']['data']

     # ค่าเริ่มและหยุดในการกวาดมุมยกแรก i=0
    i = 0
    i1 = swstart[i]
    i2 = swend[i]

    # มุมอซิมัท
    azi = v['azimuth']['data']
    az = azi[i1:i2]

    # ระยะห่างระหว่าง gate/bin
    r = v['range']['data']
    rg =r[i1:i2]

    # ค่ามุมยก
    el = v['fixed_angle']['data']
    elev=el[i] #i = 0 มุมแรก
    
    return az, rg, elev, radar_location

def save_as_geotiff(data, x, y, proj_utm, output_path):
    # Convert from UTM to WGS84
    wgs84 = wrl.georef.epsg_to_osr(4326)
    x_wgs, y_wgs = wrl.georef.reproject(x, y, projection_source=proj_utm, projection_target=wgs84)
    
    # Set up grid
    xgrid = np.linspace(x_wgs.min(), x_wgs.max(), len(x))
    ygrid = np.linspace(y_wgs.min(), y_wgs.max(), len(y))
    grid_coords = wrl.util.gridaspoints(ygrid, xgrid)
    trg_grd = np.reshape(grid_coords, (len(x), len(y), 2))
    
    # Create and save raster dataset
    depth_rt = wrl.georef.raster.create_raster_dataset(data.reshape(len(x), len(y)), trg_grd)
    wrl.io.write_raster_dataset(output_path, depth_rt, "GTiff")

# คำนวณค่า coord โดยใช้ฟังก์ชั่น ส่วนสถานีที่มีปัญหาโมเสคแล้วผลไม่มา จะใช้ค่าพารามิเตอร์ coord ที่คำนวณไว้ล่วงหน้า
def cal_qi(rg, az, coord, depth, grid_coords, CBB, lon, lat, src_crs, trg_crs):
    try:
        #print("Input shapes:")
        #print(f"rg: {rg.shape}, az: {az.shape}, coord: {coord.shape}")
        #print(f"depth: {depth.shape}, grid_coords: {grid_coords.shape}")
        #print(f"CBB: {CBB.shape}, lon: {lon.shape}, lat: {lat.shape}")
        
        # Derive quality information - pulse volume
        pulse_volume = wrl.qual.pulse_volume(rg, 1000.0, 1.0)
        pulse_volumes = np.tile(pulse_volume, az.shape[0])
        #print(f"Initial pulse_volumes shape: {pulse_volumes.shape}")
        
        # Create source coordinates for interpolation
        src_coords = np.column_stack((np.repeat(np.arange(az.shape[0]), rg.shape[0]),
                                      np.tile(np.arange(rg.shape[0]), az.shape[0])))
        # Create target coordinates
        trg_coords = np.column_stack((np.repeat(np.linspace(0, az.shape[0]-1, coord.shape[0]//rg.shape[0]), rg.shape[0]),
                                      np.tile(np.arange(rg.shape[0]), coord.shape[0]//rg.shape[0])))
        
        # ประมาณค่า pulse_volumes ให้มีขนาดที่ตรงกับของ  coord 
        pulse_volumes_interpolated = wrl.ipol.interpolate(src_coords, trg_coords, pulse_volumes, wrl.ipol.Idw, nnearest=4)
        #print(f"Interpolated pulse_volumes shape: {pulse_volumes_interpolated.shape}")
        
        # ประมาณค่า depth ให้มีขนาดที่ตรงกับของ  coord 
        depth_flat = depth.ravel()
        depth_interpolated = wrl.ipol.interpolate(src_coords, trg_coords, depth_flat, wrl.ipol.Idw, nnearest=4)
        #print(f"Interpolated depth shape: {depth_interpolated.shape}")        

        # interpolate polar radar-data and quality data to the grid
        quality_gridded = wrl.comp.togrid(
            coord,
            grid_coords,
            rg.max() + 500.0,
            coord.mean(axis=0),
            pulse_volumes_interpolated,
            wrl.ipol.Idw,
        )
        #print(f"quality_gridded shape: {quality_gridded.shape}")        

        gridded = wrl.comp.togrid(
            coord,
            grid_coords,
            rg.max() + 500.0,
            coord.mean(axis=0),
            depth_interpolated,
            wrl.ipol.Idw,
        )
        #print(f"gridded shape: {gridded.shape}")
        
        #print("....> c:11..3")
        # Transform geographic coordinates of CBB to UTM
        src_coords = np.vstack((lon.ravel(), lat.ravel())).transpose()
        utm_coords = wrl.georef.reproject(src_coords, src_crs=src_crs, trg_crs=trg_crs)
        
        # Ensure trg_coords have the same shape as utm_coords
        trg_coords = coord[..., :2].reshape(-1, 2)        

        #print(f"src_coords shape: {src_coords.shape}")
        #print(f"utm_coords shape: {utm_coords.shape}")
        #print(f"trg_coords shape: {trg_coords.shape}")
        
        if trg_coords.shape[0] != utm_coords.shape[0]:
            print(f"Warning: Mismatch in number of source and target points. Adjusting trg_coords.")
            trg_coords = trg_coords[:utm_coords.shape[0], :]
        
        #print(f"CBB shape: {CBB.shape}")
        #print(f"CBB.ravel() shape: {CBB.ravel().shape}")
        
        if CBB.ravel().shape[0] != utm_coords.shape[0]:
            print(f"Warning: Mismatch in CBB size. Adjusting CBB.")
            CBB = CBB.ravel()[:utm_coords.shape[0]]
        
        #print("....> c:11..6")
        cbb_interpolated = wrl.ipol.interpolate(utm_coords, trg_coords, CBB, wrl.ipol.Idw, nnearest=min(4, utm_coords.shape[0]))
        #print(f"cbb_interpolated shape: {cbb_interpolated.shape}")
        
        cbb_gridded = wrl.comp.togrid(
            coord,
            grid_coords,
            rg.max() + 500.0,
            coord.mean(axis=0),
            cbb_interpolated,
            wrl.ipol.Nearest,
        )
        #print(f"cbb_gridded shape: {cbb_gridded.shape}")
        #print("....> c:11..7")
        
        # Combine pulse volume and CBB for final quality index
        quality_gridded = 1 / (quality_gridded + 0.001) * (1 - cbb_gridded)
        #print(f"Final quality_gridded shape: {quality_gridded.shape}")
       
        #print("....> c:11..8")
        return (quality_gridded, gridded)
    
    except Exception as e:
        print(f"Error in cal_qi function: {str(e)}")
        print(f"Error occurred at line: {sys.exc_info()[-1].tb_lineno}")
        raise  # Re-raise the original exception if the correction fails

# คำนวณค่าพิกัดเรดาร์ด้วยการแปลงจากโพลาร์ไปเป็นกริด
proj_utm = wrl.georef.epsg_to_osr(32647)
def cal_coord(rg, az, elev, radar_location, proj_utm):
    coord = wrl.georef.spherical_to_centroids(rg, az, elev, radar_location, proj=proj_utm)
    coord = coord[..., 0:2]
    coord = coord.reshape(-1, coord.shape[-1])
    
    return coord

#คำนวณค่าพิกัดขอบเขตการโมเสค ตามพิกัดเรดาร์
def bbox(*args):
    """Get bounding box from a set of radar bin coordinates"""
    x = np.array([])
    y = np.array([])
    for arg in args:
        x = np.append(x, arg[:, 0])
        y = np.append(y, arg[:, 1])
        #print( ': ', x.min(), x.max(), ': ', y.min(), y.max())
    xmin = x.min()
    xmax = x.max()
    ymin = y.min()
    ymax = y.max()

    return xmin, xmax, ymin, ymax

# คำนวณค่ากริดผลลัพธ์ จากขอบเขตที่ได้จาก bbox ฟังก์ชัน ของทุกเรดาร์
def cal_grid_out(coord_all, num_xy):
    # define target grid for composition
    #xmin, xmax, ymin, ymax = bbox(coord_all)    
    # ฟังก์ชั่นนี้จะใช้วิธีการ fix ค่าพิกัด extent ของพื้นที่การโมเสคทั้งประเทศ เพราะต้องการจะทำให้ทุกช่วงเวลาของการกวาด มีขอบเขตที่ตรงกัน
    # ค่า xmin, xmax, ymin, ymax ได้มาจากการคำนวณหาค่า ด้วยการใช้เรดาร์ เชียงราย ลำพูน พิษณุโลก ชัยนาท เพื่อหาขอบเขต เหนือ ใต้ ออก ตก
    xmin, xmax, ymin, ymax= 150000.00, 925000.00, 1420000.00, 2500000.00
    print('>>>>>xmin, xmax, ymin, ymax of cal_grid_out: ',  xmin, xmax, ymin, ymax)
    x = np.linspace(xmin, xmax + 1000.0, num_xy)
    y = np.linspace(ymin, ymax + 1000.0, num_xy)
    grid_coords = wrl.util.gridaspoints(y, x)
    
    return x, y, grid_coords

#mos_time='2018071711'  #<---ใส่ชั่วโมงที่ต้องการทำฝนสะสมลงไป
#min_scan = ['00', '15', '30', '45']

def process_single_hour(mos_time, input_path, output_path, proj_utm, num_grid_new, x, y):
    min_scan = ['00', '15', '30', '45']
    
    # Check files for mosaic
    fd_run, files_total = check_files_mos(input_path, mos_time)
    data_gridded_hr = np.empty((len(min_scan), num_grid_new))
    
    # Load pre-calculated variables
    vars_folder = "0variables_mosaic"
    coord_list = np.load(os.path.join(vars_folder, 'coord_list.npy'), allow_pickle=True)
    coord_all = np.load(os.path.join(vars_folder, 'coord_all.npy'))
    grid_coords = np.load(os.path.join(vars_folder, 'grid_coords.npy'))
    num_xy = np.load(os.path.join(vars_folder, 'num_xy.npy'))[0]
    print("Variables loaded successfully")

    # ลูปใหญ่ ทำการโมเสคราย 15 นาที จำนวน 4 ครั้ง แล้วค่อยนำมาเก็บในอาเรย์ data_gridded_hr เพื่อ sum หา accumulate hour rain depth
    for i_s,min_s in enumerate(min_scan):

        #ตรวจจำนวนสถานี ชื่อไฟล์ที่จะใช้โมเสคแต่ละ15นาที ชื่อสถานีที่มีข้อมูล
        num_sta_available, name_sta_available, list_name_bz2_sta_available=check_radar_each15min(mos_time, files_total, min_s)
        print('\n')
        print(f'*****************\n')
        print(f'*  '+mos_time+min_s+f'  *\n')
        print(f'*****************\n')

        sta=name_sta_available
        fn_in=list_name_bz2_sta_available
        fn_out = [item + '_SNR.nc' for item in sta]    
        coord_all = np.empty(shape=(0, 2), dtype='object') # สร้างอาเรย์เพื่อเก็บค่าพิกัดของแต่ละสถานี

        #1. กรอง noise + flare ใน pyart
        print('>>1. กรอง noise + flare ใน pyart...')
        sta_fail_names=[]
        for ii_s, sta_n in enumerate(sta):    
            file_in= input_path+sta_n+'/'+fn_in[ii_s]
            file_out= input_path+sta_n+'/'+fn_out[ii_s]
            print('--> noise+flare removing : ', file_in)

            # ฟังก์ชันกรอง noise+flare ใน pyart แล้วเซฟเป็นไฟล์ใหม่เพื่อนำไปใช้ใน wradlib
            try:
                remove_noise_flare_SNR(file_in, file_out)
            except:
                print('fail: ', file_in)
                sta_fail_names.append(sta_n)
                pass


        # ตรวจสอบสถานีที่เหลืออยู่ เพื่อนำไปใช้ใน sta ลูปในกระบวนการต่อไป
        sta_remain_names = [item for item in sta if item not in sta_fail_names]
        sta = sta_remain_names
        fn_out = [item + '_SNR.nc' for item in sta]

        ### >> coord_all สำหรับแต่ละช่วงเวลาการกวาด ไม่เท่ากัน เพราะจำนวนเรดาร์ที่ใช้ไม่เท่ากัน ดังนั้นต้องทำการ fix ค่า coord_all รายชั่วโมงเลย    
        #2. คำนวณค่า extent ของเรดาร์ทุกสถานี เพื่อสร้างกริด utm ผลลัพธ์
        print('\n\n2.************************************************\n')
        print('คำนวณค่า extent ของข้อมูลเรดาร์ทุกสถานี\n')

        for i_ss, sta_n in enumerate(sta):

            print(input_path+sta_n+'/'+fn_out[i_ss])

            try:    
                file= input_path+sta_n+'/'+fn_out[i_ss]
                #อ่าน vol เข้ามา
                vol=read_generic_netcdf(file)
                #รับค่าพารามิเตอร์เพื่อคำนวณพิกัดจากฟังก์ชั่น
                az, rg, elev, radar_location = get_geom(vol, 0) #el=0 เท่ากับมุมยกที่ 1 
                #แปลงค่าพิกัดโพลาร์ เป็น กริด utm เพื่อให้ได้  coord แต่ละสถานี ที่มีสองหลัก  
                proj_utm = wrl.georef.epsg_to_osr(32647)
                coord =cal_coord(rg, az, elev, radar_location, proj_utm)        
            except:
                pass

            coord_all = np.append(coord_all, coord, axis=0)        

        # คำนวณค่ากริดผลลัพธ์ จากขอบเขตที่ได้จาก bbox ฟังก์ชัน ของทุกเรดาร์ เพื่อใช้ในการโมเสคในระบบกริด utm
        x, y, grid_coords = cal_grid_out(coord_all, num_xy)
        data_qi_gridded = np.empty((len(sta), num_grid_new))  #1000000 เป็นค่า  quality_gridded.shape
        data_gridded = np.empty((len(sta), num_grid_new))  #1000000 เป็นค่า  quality_gridded.shape   

        #3. คำนวณค่า rainfall depth และ Quality index  ของแต่ละสถานีในระบบโพลาร์
        print('\n\n 3.************************************************\n')
        print('คำนวณค่า rainfall depth และ Quality index  ของแต่ละสถานีในระบบโพลาร์\n')
        data_list = []
        quality_list = []

        sta_fail_indices=[]
        sta_fail_names=[]

        for i, sta_n in enumerate(sta):
            print(input_path+sta_n+'/'+fn_out[i])
            file = input_path + sta_n + '/' + fn_out[i]

            try:
                vol = read_generic_netcdf(file)   
                az, rg, elev, radar_location = get_geom(vol, 0)        
                depth = correct_clutter_attenuation(vol, 0)     
                proj_utm = wrl.georef.epsg_to_osr(32647)        
                #coord = coord_list[i] # ทดลองคอมเม้นท์ไม่เอาค่าที่คำนวณไว้เอง แล้วไปคำนวณ coord เองจากฟังก์ชั่น อาจจะช้าหน่อย --> ผลลัพธ์ ไม่ได้ช้า เวลา 00 ทำได้ครบทุก4สถานี แต่เวลา 15 ไม่มีLMPในผลลัพธ์ ทั้งที่มีไฟล์
                coord = cal_coord(rg, az, elev, radar_location, proj_utm)

                print(f"\nProcessing Radar {i + 1}")
                #print(f"Depth - Min: {np.min(depth)}, Max: {np.max(depth)}, Mean: {np.mean(depth)}")
                cbb_file = os.path.join("cbb_outputs", f"{sta_n}_CBB.npy")      
                CBB = np.load(cbb_file)      
                #print(f"Loaded CBB shape: {CBB.shape}")
                # Extract longitude and latitude for CBB
                lon, lat = coord[..., 0], coord[..., 1]
                # Define source and target projections
                src_crs = wrl.georef.epsg_to_osr(4326)  # WGS84 geographic coordinates
                trg_crs = proj_utm  # UTM coordinates      
                # Calculate quality index with CBB interpolation
                quality_gridded, data_gridded = cal_qi(rg, az, coord, depth, grid_coords, CBB, lon, lat, src_crs, trg_crs)
                # if fail ให้คำนวณ coord เอง       
                print(f"Data gridded - Min: {np.nanmin(data_gridded)}, Max: {np.nanmax(data_gridded)}, Mean: {np.nanmean(data_gridded)}")
                data_list.append(data_gridded)        
                quality_list.append(quality_gridded)
            except Exception as e:
                print(f"Error processing station {sta_n}: {str(e)}")
                print(f"Error occurred at line: {sys.exc_info()[-1].tb_lineno}")        
                # Attempt to use pre-calculated coordinates
                try:
                    print(f"Attempting to use pre-calculated coordinates for station {sta_n}...")
                    coord = coord_list[i]

                    # Extract longitude and latitude for CBB
                    lon, lat = coord[..., 0], coord[..., 1]

                    # Recalculate quality index with CBB interpolation using pre-calculated coordinates
                    quality_gridded, data_gridded = cal_qi(rg, az, coord, depth, grid_coords, CBB, lon, lat, src_crs, trg_crs)

                    print(f"Processing successful with pre-calculated coordinates for station {sta_n}")
                    print(f"Data gridded - Min: {np.nanmin(data_gridded)}, Max: {np.nanmax(data_gridded)}, Mean: {np.nanmean(data_gridded)}")
                    data_list.append(data_gridded)
                    quality_list.append(quality_gridded)
                except Exception as new_e:
                    print(f"Error in using pre-calculated coordinates for station {sta_n}: {str(new_e)}")
                    sta_fail_indices.append(i)
                    sta_fail_names.append(sta_n)
                    print('fail: ', i, sta_n)
                continue

        # ตรวจสอบสถานีที่เหลืออยู่ กับ อินเด็กซ์เก่าที่เหลืออยู่ เพื่อนำไปใช้ composite
        sta_remain_names = [item for item in sta if item not in sta_fail_names]
        sta_remain_indices = [index for index, item in enumerate(sta) if item not in sta_fail_names]

        print('fail sta: ', sta_fail_names) # ต้องเขียนออกไป เป็นไฟล์รายงาน ว่าการโมเสคครั้งนี้ มีไฟล์อะไรบ้างที่เปิดไม่ได้ หรือ มีปัญหากี่ไฟล์ ชื่ออะไรบ้าง 
        print('remaining sta: ', sta_remain_names) # ต้องเขียนออกไป เป็นไฟล์รายงาน ว่าการโมเสคครั้งนี้ ใช้ไฟล์กี่ไฟล์ ชื่ออะไรบ้าง 
        print('remaining sta indices: ', sta_remain_indices)

        print('\n\n 4.************************************************\n')
        print('ทำแผนที่ composite / mosaic\n')

        '''
        for i in range(len(data_list)):
            data_list[i][data_list[i] < 0] = np.nan
            data_list[i][data_list[i] > 100] = np.nan
        '''
        composite = wrl.comp.compose_weighted(data_list, quality_list)
        composite[composite < 0] = np.nan
        composite[composite > 100] = np.nan
        composite = np.ma.masked_invalid(composite)


        print(f"Composite - Shape: {composite.shape}, Non-NaN count: {np.sum(~np.isnan(composite))}")
        print(f"Composite - Min: {np.nanmin(composite)}, Max: {np.nanmax(composite)}, Mean: {np.nanmean(composite)}")

        #-เก็บแผนที่ฝน composite ราย 15 นาที ไว้ในอาเรย์ฝนรายชั่วโมง เพื่อไปคำนวณฝนสะสม
        data_gridded_hr[i_s] = composite

        ##############################################
        #-5.ทำรายงานจำนวนสถานีและไฟล์ที่ใช้โมเสคในช่วงเวลานี้
        print('\n\n 5.************************************************\n')
        print('ทำรายงานจำนวนสถานีและไฟล์ที่ใช้โมเสคในช่วงเวลานี้\n')
        num_sta_used_mosaic=len(sta_remain_names)

        #เวลา, จำนวนสถานีที่มีข้อมูล, ชื่อสถานีที่มีข้อมูล, จำนวนสถานีที่ที่ใช้ในการโมเสค, ชื่อสถานีที่ที่ใช้ในการโมเสค, ชื่อสถานีที่อ่านค่าไม่ได้
        report=[mos_time + min_s, num_sta_available,name_sta_available,num_sta_used_mosaic,sta_remain_names,sta_fail_names]

        # Convert to Thailand time only for saving
        thailand_time = convert_utc_to_thailand_for_saving(mos_time)
        
        # เขียนไฟล์ที่ใช้แต่ละการกวาด ลงในโหมด append 
        # ตรวจสอบไฟล์มีอยู๋หรือไม่Check if the file exists
        output_file_path = output_path+'data_used_'+thailand_time+'.txt'

        #เขียนรายงานผลการใช้เรดาร์ในการโมเสคแต่ละช่วงเวลา
        write_report(mos_time, report, i_s, output_file_path)    

        

    #จบลูปใหญ่ หลังจากคำนวณ 4 โมเสคไฟล์ตรงนี้
    # รวมฝนสะสมรายชั่วโมง

    #accum_hr = np.nansum(data_gridded_hr, axis=0)          #รวมอย่างง่าย แต่ผลลัพธ์จะขาดสถานีที่ไม่มีข้อมูลครบ 4 ครั้ง
   
    # Calculate hourly accumulation ใช้ Weighted Summation 
    valid_counts = np.sum(~np.isnan(data_gridded_hr), axis=0)
    accum_hr = np.nansum(data_gridded_hr, axis=0) * (4 / valid_counts)
    
    # Calculate frequency
    frequency = np.sum(~np.isnan(data_gridded_hr), axis=0)
    mask_sta_used = np.where(frequency >= 1, 1, -1)
    
    #print('\n<<<<<<<<<<<************************************************>>>>>>>>>>\n')
    return accum_hr, frequency, mask_sta_used

# คำนวณค่ากริดผลลัพธ์ จากขอบเขตที่ได้จาก bbox ฟังก์ชัน ของทุกเรดาร์
def cal_grid_out_not_used(coord_all, num_xy):
    # define target grid for composition
    xmin, xmax, ymin, ymax = bbox(coord_all) 
    #x = np.linspace(xmin, xmax + 1000.0, 1000)
    #y = np.linspace(ymin, ymax + 1000.0, 1000)
    x = np.linspace(xmin, xmax + 1000.0, num_xy)
    y = np.linspace(ymin, ymax + 1000.0, num_xy)
    grid_coords = wrl.util.gridaspoints(y, x)
    
    return x, y, grid_coords

def check_files_mos(path, mos_time): 
    print('\n\n--->โมเสคทั้งหมด แม้ว่าจะมีไม่ครบ 4 ครั้ง/ชั่วโมง')
    
    # ตรวจสอบไฟล์ gzip เพื่อแตกไฟล์เป็น uf
    check_and_decompress(path, mos_time)

    # ค้นหาสถานีที่มีการสแกน แม้ว่าจะมีไม่ครบ 4 ครั้ง/ชั่วโมง
    fd_name = os.listdir(path)
    numFiles=[]
    fd_run=[]
    for i, fd in enumerate(fd_name):
        j=0
        for file in os.listdir(path+'/'+ fd):
            if file.endswith('.uf'):
                j+=1            
        if j!=0 : 
            numFiles.append(j)
            fd_run.append(fd)

    #-ลิสต์ไฟล์ตามเวลาที่สแกน เพื่อนำไปโมเสคราย 15 นาที

    min_fn = ["00", "15", "30", "45"] #สแกนนาที ที่ [00, 15, 30, 45]
    files_total=[]

    for min_s in min_fn:
        files_total.append('scan minute: '+ min_s)
        j=0
        files=[]
        fd_n=[]
        for i, fd in enumerate(fd_run):
            #print(fd+'---')        
            for file in os.listdir(path+'/'+ fd):
                if file.endswith('.uf') and re.search(r"2018(\d{6})"+min_s, file) :
                #if file.endswith('.bz2') and re.search(r"2018(\d{6})"+min_s, file) :
                    #print(i, fd, file)
                    files.append(file) #เพิ่มเฉพาะไฟล์ที่เจอในลิสต์ 
                    fd_n.append(fd) #เพิ่มเฉพาะโฟลเดอร์ที่เจอ *uf ในลิสต์
                    j+=1
                    #print(file)     
        files_total.append(j)
        files_total.append(fd_n)
        files_total.append(files)
    
    return fd_run, files_total



def check_radar_each15min(mos_time, files_total, min_s):
    #ตรวจสอบค่าพารามิเตอร์ที่จะใช้ในการโมเสคราย 15 นาที
   
    scan_time=min_s

    # Iterate through files_total to find the index of the matching scan minute
    index = None
    for i, item in enumerate(files_total):
        if isinstance(item, str) and item == f'scan minute: {scan_time}':
            index = i
            break

    # Check if a match was found and print the index
    if index is not None:
        num_sta_available=files_total[index+1]
        name_sta_available=files_total[index+2]
        list_name_bz2_sta_available=files_total[index+3]
    else:
        print(f"'scan minute: {scan_time}' not found in the list")
    
    
    return num_sta_available, name_sta_available, list_name_bz2_sta_available

def write_report(mos_time, report, i, output_file_path):
   
    # Check if the file exists and i is 0
    if os.path.exists(output_file_path) and i == 0:
        # If it exists and i is 0, delete it
        os.remove(output_file_path)
        # Create a new blank file
        with open(output_file_path, 'w') as fp:
            fp.write("")

    # Open the file in append mode and write the report
    with open(output_file_path, 'a') as fp:
        # Write the contents for the current scanning time
        for item in report:
            fp.write(f"{item}\n")
            
def main():
    
    # อ่านชื่อไฟล์ชั่วโมงที่ต้องการเข้ามาในลิสต์ ไฟล์ดังกล่าวสร้างจากฟังก์ชั่น extract_timestamps, save_timestamp_list
    #mos_times = ['2018071711', '2018071712', '2018071713']
    mos_times = read_timestamp_list('./0variables_mosaic/mos_hourly_times.txt')
    
    # ลบไฟล์ uf กับ nc ทิ้ง เพื่อเคลียร์พื้นที่ ก่อนการโมเสค
    logging.info("Starting radar data cleanup process")
    cleanup_files(input_path)
    logging.info("Radar data cleanup process completed")
    
    # Load necessary variables
    vars_folder = "0variables_mosaic"
    x = np.load(os.path.join(vars_folder, 'x.npy'))
    y = np.load(os.path.join(vars_folder, 'y.npy'))
    num_grid_new = len(x) * len(y)
    
    proj_utm = wrl.georef.epsg_to_osr(32647)  # UTM zone 47N    
    for mos_time in mos_times:
        cleanup_files(input_path) # ลบไฟล์ uf กับ nc ทิ้ง เพื่อเคลียร์พื้นที่ ก่อนการโมเสค
        print(f"\nProcessing mosaic for hour: {mos_time}")
        accum_hr, frequency, mask_sta_used = process_single_hour(mos_time, input_path, output_path, proj_utm, num_grid_new, x, y)
        
        # Convert to Thailand time only for saving
        thailand_time = convert_utc_to_thailand_for_saving(mos_time)
        
        # Save hourly accumulation as GeoTIFF
        accum_output_path = os.path.join(output_path, f'{thailand_time}_2000m.tif')
        save_as_geotiff(accum_hr, x, y, proj_utm, accum_output_path)
        print(f"Hourly accumulation saved as: {accum_output_path}") 
        
        # Save frequency as GeoTIFF
        freq_output_path = os.path.join(output_path, f'{thailand_time}_frequency.tif')
        save_as_geotiff(frequency, x, y, proj_utm, freq_output_path)
        print(f"Frequency data saved as: {freq_output_path}")
        
        # ลบไฟล์ uf กับ nc ทิ้ง เพื่อเคลียร์พื้นที่ หลังการโมเสค
        print(f"Deleted extracted files *uf & *nc for hour: {mos_time}")
        logging.info("Starting radar data cleanup process")
        cleanup_files(input_path)
        logging.info("Radar data cleanup process completed")
        
        
        print(f"Completed processing for hour: {mos_time}")
        print('\n<<<<<<<<<<<************************************************>>>>>>>>>>\n')
        
        

        


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

1.14.0
3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 05:59:45) [MSC v.1929 64 bit (AMD64)]
0.0.0


# ===main program===

### ตั้งค่าผลิตภัณฑ์ฝนตรงนี้

In [2]:
#>>> ตั้งค่าผลิตภัณฑ์ฝนตรงนี้
#เปลี่ยนขนาดกริดตรงนี้
gridsize_new=2000 #ขนาดกริดใหม่ หน่วยเมตร ต้องไปเปลี่ยนใน "11คำนวณตัวแปรในการโมเสคล่วงหน้า.ipynb" ด้วย
gridsize_org = 1000 #ขนาดกริดตั้งต้น
num_grid_org = gridsize_org**2 #จำนวนกริดตั้งต้น
num_xy = int(num_grid_org/gridsize_new) #จำนวนกริดในแนวแกน x และy
num_grid_new = int(num_grid_org/gridsize_new)**2 #จำนวนกริดใหม่ทั้งหมด.

# >>> ตั้งค่าโฟลเดอร์ข้อมูล กับ ผลลัพธ์การโมเสค
#Input path
input_path = '../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/'

#Output path
output_path = './zProcessing_temp/0hourly_CBB_output/'

### รัน main โมเสคฝนรายชั่วโมง

In [3]:
if __name__ == "__main__":
    main()


Processing mosaic for hour: 2018071711


--->โมเสคทั้งหมด แม้ว่าจะมีไม่ครบ 4 ครั้ง/ชั่วโมง
Decompressing ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171100.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171100.uf
Decompressing ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171115.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171115.uf
Decompressing ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171130.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171130.uf
Decompressing ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171145.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/CHN\CHN240@201807171145.uf
Decompressing ../1data/0radar/0Mosaic_MultiHourly_Sentihn/0_3hours/C

### คำนวณเวลาในการโมเสค

In [4]:
# wait for 3 seconds
time.sleep(3)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('เวลาที่ใช้โมเสค:', elapsed_time, 'seconds')

เวลาที่ใช้โมเสค: 1220.5998344421387 seconds


# ลบไฟล์ที่ได้ระหว่างกระบวนการ เพื่อลดเนื้อที่

In [5]:
# ลบไฟล์ที่ได้ระหว่างกระบวนการ เพื่อลดเนื้อที่
cleanup_files(input_path)